In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import skfuzzy as fuzz
from scipy.stats import mode
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
data_dict = {
    'Age': [25, 34, 45, 23, 35, 52, 46, 51, 27, 40],
    'Income': [30, 45, 60, 25, 50, 80, 65, 75, 35, 55],
    'SpendingScore': [60, 70, 40, 80, 65, 30, 45, 35, 75, 50]
}
df = pd.DataFrame(data_dict)
df


,Age,Income,SpendingScore
0,25,30,60
1,34,45,70
2,45,60,40
3,23,25,80
4,35,50,65
5,52,80,30
6,46,65,45
7,51,75,35
8,27,35,75
9,40,55,50


In [9]:
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(df)


In [ ]:

df = pd.DataFrame(data_dict)
df


In [4]:
data = X_norm.T  # shape: features x samples

cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data, c=3, m=2, error=0.005, maxiter=1000, init=None)

predicted_clusters = np.argmax(u, axis=0)
df['FCM_Cluster'] = predicted_clusters
df


,Age,Income,SpendingScore,FCM_Cluster
0,25,30,60,0
1,34,45,70,2
2,45,60,40,1
3,23,25,80,0
4,35,50,65,2
5,52,80,30,1
6,46,65,45,1
7,51,75,35,1
8,27,35,75,0
9,40,55,50,2


In [5]:
print("Membership Matrix (u):\n", np.round(u.T, 2))
print("\nCluster Centers:\n", np.round(cntr, 2))


Membership Matrix (u):
 [[0.76 0.04 0.2 ]
 [0.11 0.02 0.87]
 [0.04 0.85 0.11]
 [0.92 0.01 0.07]
 [0.   0.   0.99]
 [0.03 0.92 0.06]
 [0.03 0.88 0.09]
 [0.01 0.97 0.02]
 [0.93 0.01 0.06]
 [0.1  0.31 0.59]]

Cluster Centers:
 [[0.07 0.1  0.86]
 [0.88 0.82 0.15]
 [0.43 0.43 0.68]]


In [6]:
for i, center in enumerate(cntr):
    age, income, score = center
    print(f"\nCluster {i}: Age={age:.2f}, Income={income:.2f}, Spending Score={score:.2f}")
    if income > 0.6 and score < 0.4:
        print("→ High income, low spending")
    elif income < 0.4 and score > 0.6:
        print("→ Low income, high spending")
    else:
        print("→ Moderate profile")



Cluster 0: Age=0.07, Income=0.10, Spending Score=0.86
→ Low income, high spending

Cluster 1: Age=0.88, Income=0.82, Spending Score=0.15
→ High income, low spending

Cluster 2: Age=0.43, Income=0.43, Spending Score=0.68
→ Moderate profile


In [7]:
kmeans = KMeans(n_clusters=3, random_state=42).fit(X_norm)
df['KMeans_Cluster'] = kmeans.labels_
df


,Age,Income,SpendingScore,FCM_Cluster,KMeans_Cluster
0,25,30,60,0,0
1,34,45,70,2,2
2,45,60,40,1,1
3,23,25,80,0,0
4,35,50,65,2,2
5,52,80,30,1,1
6,46,65,45,1,1
7,51,75,35,1,1
8,27,35,75,0,0
9,40,55,50,2,1


In [8]:
mapping_kmeans = {}
for cluster_id in range(3):
    mask = df['KMeans_Cluster'] == cluster_id
    if np.any(mask):
        mapping_kmeans[cluster_id] = mode(df['FCM_Cluster'][mask], keepdims=True).mode[0]

mapped_kmeans = df['KMeans_Cluster'].map(mapping_kmeans)
accuracy_kmeans = np.mean(mapped_kmeans == df['FCM_Cluster'])

print("\nFCM vs K-means Cluster Agreement Accuracy:", accuracy_kmeans)
print("FPC Value:", fpc)
print("→ FPC closer to 1 means better cluster separation. FCM handles overlap better than K-means.")



FCM vs K-means Cluster Agreement Accuracy: 0.9
FPC Value: 0.7849816870103827
→ FPC closer to 1 means better cluster separation. FCM handles overlap better than K-means.
